In [1]:
import pandas as pd
!pip install yfinance
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, Bounds, LinearConstraint

In [9]:
start_date = "2017-01-03"
end_date = "2022-07-29"
tickerlist = ['RY.TO','TD.TO','BNS.TO','CM.TO','CWB.TO','HCG.TO','LB.TO','NA.TO','BMO.TO']

In [10]:
tickerstr = ''.join(tickerlist)
data = yf.download(tickers=tickerlist, start=start_date, end=end_date, interval="1d", group_by='ticker')

[*********************100%***********************]  9 of 9 completed


In [8]:
print(data)

                LB.TO                                                      \
                 Open       High        Low      Close  Adj Close  Volume   
Date                                                                        
2017-01-03  57.990002  58.840000  57.980000  58.310001  43.809845  106400   
2017-01-04  58.230000  58.880001  57.900002  58.700001  44.102867  107600   
2017-01-05  58.799999  59.169998  58.549999  58.790001  44.170483   61600   
2017-01-06  58.750000  58.840000  58.450001  58.689999  44.095348   44800   
2017-01-09  58.500000  58.939999  58.230000  58.650002  44.065300   62200   
...               ...        ...        ...        ...        ...     ...   
2022-07-21  41.169998  41.410000  40.980000  41.299999  41.299999  113800   
2022-07-22  41.389999  41.720001  41.240002  41.540001  41.540001  182900   
2022-07-25  41.730000  42.320000  41.560001  42.180000  42.180000  211800   
2022-07-26  42.000000  42.349998  41.509998  41.529999  41.529999  181800   

In [11]:
df=pd.DataFrame(index=data.index)
for x in tickerlist:
    df = pd.merge(df, data[x]['Adj Close'].rename(x), left_index=True, right_index=True)

df=df.dropna(axis=0)

In [12]:
print(df)

                 RY.TO      TD.TO     BNS.TO      CM.TO     CWB.TO     HCG.TO  \
Date                                                                            
2017-01-03   73.316238  53.386005  58.153591  42.166840  25.346199  31.204655   
2017-01-04   74.389961  53.976433  58.644836  42.478794  25.766714  31.263548   
2017-01-05   74.373917  54.104084  58.775322  42.535858  25.717239  30.988705   
2017-01-06   74.293793  54.176483  58.790668  42.490208  25.741980  30.704044   
2017-01-09   74.133553  53.927128  58.606457  42.353256  25.387428  29.977671   
...                ...        ...        ...        ...        ...        ...   
2022-07-22  122.339996  82.029999  75.089996  62.959999  25.299999  25.150000   
2022-07-25  123.099998  82.470001  75.739998  63.290001  25.600000  25.620001   
2022-07-26  121.849998  81.400002  75.860001  63.049999  25.209999  25.190001   
2022-07-27  122.650002  82.440002  76.699997  63.680000  25.389999  25.540001   
2022-07-28  123.750000  82.0

In [13]:
df_pctchg = df.pct_change()

In [14]:
print(df_pctchg)

               RY.TO     TD.TO    BNS.TO     CM.TO    CWB.TO    HCG.TO  \
Date                                                                     
2017-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2017-01-04  0.014645  0.011060  0.008447  0.007398  0.016591  0.001887   
2017-01-05 -0.000216  0.002365  0.002225  0.001343 -0.001920 -0.008791   
2017-01-06 -0.001077  0.001338  0.000261 -0.001073  0.000962 -0.009186   
2017-01-09 -0.002157 -0.004603 -0.003133 -0.003223 -0.013773 -0.023657   
...              ...       ...       ...       ...       ...       ...   
2022-07-22 -0.003707 -0.002553 -0.003451  0.004147 -0.009397  0.003992   
2022-07-25  0.006212  0.005364  0.008656  0.005241  0.011858  0.018688   
2022-07-26 -0.010154 -0.012974  0.001584 -0.003792 -0.015234 -0.016784   
2022-07-27  0.006565  0.012776  0.011073  0.009992  0.007140  0.013894   
2022-07-28  0.008969 -0.004488  0.006519  0.007538  0.003939  0.011355   

               LB.TO     NA.TO    BMO

In [15]:
corr = df_pctchg.corr()

In [16]:
print (corr)

           RY.TO     TD.TO    BNS.TO     CM.TO    CWB.TO    HCG.TO     LB.TO  \
RY.TO   1.000000  0.850590  0.842268  0.813709  0.606172  0.303547  0.625138   
TD.TO   0.850590  1.000000  0.865843  0.818254  0.641601  0.317292  0.658345   
BNS.TO  0.842268  0.865843  1.000000  0.834102  0.638694  0.323147  0.672760   
CM.TO   0.813709  0.818254  0.834102  1.000000  0.640523  0.333083  0.663341   
CWB.TO  0.606172  0.641601  0.638694  0.640523  1.000000  0.388507  0.627081   
HCG.TO  0.303547  0.317292  0.323147  0.333083  0.388507  1.000000  0.290011   
LB.TO   0.625138  0.658345  0.672760  0.663341  0.627081  0.290011  1.000000   
NA.TO   0.807352  0.807550  0.824888  0.808224  0.661559  0.348647  0.649697   
BMO.TO  0.835289  0.843696  0.857003  0.857002  0.664418  0.331414  0.675475   

           NA.TO    BMO.TO  
RY.TO   0.807352  0.835289  
TD.TO   0.807550  0.843696  
BNS.TO  0.824888  0.857003  
CM.TO   0.808224  0.857002  
CWB.TO  0.661559  0.664418  
HCG.TO  0.348647  0.33141